[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/basf/mlipx/blob/main/docs/source/notebooks/combine.ipynb)

# Combine Multiple Nodes

The `mlipx` command line interface provides you with recipes for specific tasks.
In this notebook, we will write a script to include different aspects from different recipes into a single workflow.

In [1]:
# Only install the packages if they are not already installed
!pip show mlipx > /dev/null 2>&1 || pip install mlipx
!pip show rdkit2ase > /dev/null 2>&1 || pip install rdkit2ase

In [2]:
# We will create a GIT and DVC repository in a temporary directory
import os
import tempfile

temp_dir = tempfile.TemporaryDirectory()
os.chdir(temp_dir.name)

Like all `mlipx` Nodes we will use a GIT and DVC repository to run experiments.
To make our custom code available, we structure our project like

```
relaxation/
   ├── .git/
   ├── .dvc/
   ├── src/__init__.py
   ├── src/relaxation.py
   ├── models.py
   └── main.py
```

to allow us to import our code `from src.relaxation import Relax`.
Alternatively, you can package your code and import it like any other Python package.

In [3]:
!git init
!dvc init --quiet

Initialized empty Git repository in /private/var/folders/81/0x90c4yd0cz6tz104v9syf580000gn/T/tmpqt0ewb9_/.git/


Let us configure now configure a workflow, creating a structure from `SMILES`, relax it, run molecular dynamics and compute the homonuclear diatomics.

In [4]:
import mlipx

project = mlipx.Project()

emt = mlipx.GenericASECalculator(
    module="ase.calculators.emt",
    class_name="EMT",
)

with project.group("initialize"):
    confs = mlipx.Smiles2Conformers(smiles="CCCC", num_confs=1)

with project.group("structure-optimization"):
    struct_optim = mlipx.StructureOptimization(
        data=confs.frames, data_id=-1, optimizer="LBFGS", model=emt
    )

thermostat = mlipx.LangevinConfig(
    timestep=0.5,
    temperature=300,
    friction=0.001,
)

with project.group("molecular-dynamics"):
    md = mlipx.MolecularDynamics(
        data=struct_optim.frames,
        data_id=-1,
        model=emt,
        thermostat=thermostat,
        steps=1000,
    )

with project.group("homonuclear-diatomics"):
    ev = mlipx.HomonuclearDiatomics(
        data=confs.frames,
        model=emt,
        n_points=100,
        min_distance=0.75,
        max_distance=2.0,
        elements=[],
    )

project.repro()

2024-12-04 08:57:41,326 - INFO: Saving params.yaml


100%|██████████| 4/4 [00:00<00:00, 721.91it/s]


Running stage 'initialize_Smiles2Conformers':
> zntrack run mlipx.nodes.smiles.Smiles2Conformers --name initialize_Smiles2Conformers
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

Running stage 'homonuclear-diatomics_HomonuclearDiatomics':
> zntrack run mlipx.nodes.diatomics.HomonuclearDiatomics --name homonuclear-diatomics_HomonuclearDiatomics


C-C bond (1.52 Å): 100%|██████████| 100/100 [00:00<00:00, 2193.13it/s]


Updating lock file 'dvc.lock'

Running stage 'structure-optimization_StructureOptimization':
> zntrack run mlipx.nodes.structure_optimization.StructureOptimization --name structure-optimization_StructureOptimization
       Step     Time          Energy          fmax
LBFGS:    0 08:57:45        4.724808        4.228024
LBFGS:    1 08:57:45        3.694983        2.695507
LBFGS:    2 08:57:45        3.002409        1.492604
LBFGS:    3 08:57:45        2.821451        1.084243
LBFGS:    4 08:57:45        2.664413        1.026595
LBFGS:    5 08:57:45        2.491459        0.799510
LBFGS:    6 08:57:45        2.425135        0.284369
LBFGS:    7 08:57:45        2.421406        0.196046
LBFGS:    8 08:57:45        2.419364        0.177538
LBFGS:    9 08:57:45        2.415200        0.195117
LBFGS:   10 08:57:45        2.410975        0.242129
LBFGS:   11 08:57:45        2.406195        0.232087
LBFGS:   12 08:57:45        2.403611        0.114803
LBFGS:   13 08:57:45        2.402815        

1001it [00:02, 338.00it/s]                         


Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add nodes/homonuclear-diatomics/HomonuclearDiatomics/.gitignore nodes/initialize/Smiles2Conformers/.gitignore nodes/structure-optimization/StructureOptimization/.gitignore nodes/molecular-dynamics/MolecularDynamics/.gitignore dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


Once the graph has been executed, we can look at the resulting structures.

In [5]:
print(md.figures.keys())
print(struct_optim.figures.keys())
print(ev.figures.keys())

dict_keys(['energy', 'fmax', 'fnorm'])
dict_keys(['energy_vs_steps', 'fmax_vs_steps'])
dict_keys(['H-H bond', 'C-C bond'])


In [6]:
temp_dir.cleanup()